In [1]:
from google.colab import files

uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Load data
df = pd.read_csv("dataset.csv")  # Replace with your file
texts = df["text"].values
labels = df["label"].values

# Fix labels (if they're strings or invalid)
label_map = {"Negative": 0, "Neutral": 1, "Positive": 2}  # Adjust based on your data
if isinstance(labels[0], str):
    labels = np.array([label_map[label] for label in labels])

# Ensure labels are integers in [0, 2]
labels = np.clip(labels.astype(int), 0, 2)

# Split data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Padding
max_len = 128
X_train = pad_sequences(train_sequences, maxlen=max_len, padding="post", truncating="post")
X_test = pad_sequences(test_sequences, maxlen=max_len, padding="post", truncating="post")

# Convert labels to categorical
y_train = to_categorical(train_labels, num_classes=3)
y_test = to_categorical(test_labels, num_classes=3)

print("Shapes:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)

Shapes:
X_train: (805, 128) y_train: (805, 3)
X_test: (202, 128) y_test: (202, 3)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model_lstm = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=128),  # Matches X_train.shape[1]
    LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    LSTM(32),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes for SentMix-3L
])

model_lstm.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
from tensorflow.keras.layers import GRU

model_gru = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=128),
    GRU(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    GRU(32),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model_gru.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
history_lstm = model_lstm.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32,
    verbose=1
)

Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 16s 334ms/step - accuracy: 0.3610 - loss: 1.0911 - val_accuracy: 0.3861 - val_loss: 1.0978
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 288ms/step - accuracy: 0.3896 - loss: 1.0926 - val_accuracy: 0.3861 - val_loss: 1.0921
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 234ms/step - accuracy: 0.3728 - loss: 1.0690 - val_accuracy: 0.3861 - val_loss: 1.0941
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 290ms/step - accuracy: 0.4115 - loss: 1.0693 - val_accuracy: 0.3861 - val_loss: 1.0940
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 294ms/step - accuracy: 0.4503 - loss: 1.0540 - val_accuracy: 0.3861 - val_loss: 1.0912
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 252ms/step - accuracy: 0.4413 - loss: 1.0558 - val_accuracy: 0.3861 - val_loss: 1.0829
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 269ms/step - accuracy: 0.4246 - loss: 1.0428 - val_accuracy: 0.3812 - val_loss: 1.0885
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 253ms/step - accuracy: 0.3925 - loss: 1.0551 - val_accuracy

In [9]:
history_gru = model_gru.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32,
    verbose=1
)

Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 17s 336ms/step - accuracy: 0.4532 - loss: 1.0780 - val_accuracy: 0.3416 - val_loss: 1.0921
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 278ms/step - accuracy: 0.3771 - loss: 1.0820 - val_accuracy: 0.3861 - val_loss: 1.0915
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 315ms/step - accuracy: 0.4133 - loss: 1.0657 - val_accuracy: 0.3861 - val_loss: 1.0948
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 319ms/step - accuracy: 0.3741 - loss: 1.0814 - val_accuracy: 0.3861 - val_loss: 1.0927
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 261ms/step - accuracy: 0.4336 - loss: 1.0534 - val_accuracy: 0.3861 - val_loss: 1.0895
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 243ms/step - accuracy: 0.4036 - loss: 1.0674 - val_accuracy: 0.3861 - val_loss: 1.0931
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 8s 314ms/step - accuracy: 0.4063 - loss: 1.0677 - val_accuracy: 0.3861 - val_loss: 1.0892
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 306ms/step - accuracy: 0.4438 - loss: 1.0596 - val_accurac

In [10]:
# Evaluate LSTM
loss_lstm, acc_lstm = model_lstm.evaluate(X_test, y_test)

# Evaluate GRU
loss_gru, acc_gru = model_gru.evaluate(X_test, y_test)

print(f"LSTM Test Accuracy: {acc_lstm:.4f}")
print(f"GRU Test Accuracy: {acc_gru:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.6057 - loss: 0.9010
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.3813 - loss: 1.0911
LSTM Test Accuracy: 0.6287
GRU Test Accuracy: 0.3861


In [11]:
model_lstm.save("lstm_sentiment.h5")
model_gru.save("gru_sentiment.h5")

In [15]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def predict_sentiment(text, model, tokenizer, max_len=128):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=max_len, padding='post', truncating='post')
    prediction = model.predict(padded)
    return np.argmax(prediction)  # Returns 0 (Negative), 1 (Neutral), 2 (Positive)

# Label mapping
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

In [17]:
sample_text = "I am happy"  # English
sample_text_mixed = "আমি happy!"  # Bangla-English

# LSTM Prediction
lstm_pred = predict_sentiment(sample_text, model_lstm, tokenizer)
print(f"LSTM Prediction ('{sample_text}'): {label_map[lstm_pred]}")

# GRU Prediction
gru_pred = predict_sentiment(sample_text, model_gru, tokenizer)
print(f"GRU Prediction ('{sample_text}'): {label_map[gru_pred]}")

# Test code-mixed text
lstm_pred_mixed = predict_sentiment(sample_text_mixed, model_lstm, tokenizer)
print(f"LSTM Prediction ('{sample_text_mixed}'): {label_map[lstm_pred_mixed]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
LSTM Prediction ('I am happy'): Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
GRU Prediction ('I am happy'): Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
LSTM Prediction ('আমি happy!'): Negative
